Elasticity is defined by the following relation known as the Hooke's law:

$$
\sigma_{ij} = C_{ijkl} \cdot \varepsilon_{kl}
$$

which expanded becomes a 6 x 6 symmetric tensor 

$$
\begin{bmatrix}
\sigma_{11} \\
\sigma_{22} \\
\sigma_{33} \\
\sigma_{23} \\
\sigma_{31} \\
\sigma_{12} 
\end{bmatrix}
=
\begin{bmatrix}
C_{1111} & C_{1122} & C_{1133} & C_{1123} & C_{1131} & C_{1112} \\
C_{2211} & C_{2222} & C_{2233} & C_{2223} & C_{2231} & C_{2212} \\
C_{3311} & C_{3322} & C_{3333} & C_{3323} & C_{3331} & C_{3312} \\
C_{2311} & C_{2322} & C_{2333} & C_{2323} & C_{2331} & C_{2312} \\
C_{3111} & C_{3122} & C_{1333} & C_{3123} & C_{3131} & C_{3112} \\
C_{1211} & C_{1222} & C_{1233} & C_{1223} & C_{1231} & C_{1212} 
\end{bmatrix}

\begin{bmatrix}
\epsilon_{11} \\
\epsilon_{22} \\
\epsilon_{33} \\
2\epsilon_{23} \\
2\epsilon_{31} \\
2\epsilon_{12} 
\end{bmatrix}
$$

which in Voigt notation becomes

$$
\begin{bmatrix}
\sigma_{1} \\
\sigma_{2} \\
\sigma_{3} \\
\sigma_{4} \\
\sigma_{5} \\
\sigma_{6} 
\end{bmatrix}
=
\begin{bmatrix}
C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
C_{12} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
C_{13} & C_{23} & C_{33} & C_{34} & C_{35} & C_{36} \\
C_{14} & C_{24} & C_{34} & C_{44} & C_{45} & C_{46} \\
C_{15} & C_{25} & C_{35} & C_{45} & C_{55} & C_{56} \\
C_{16} & C_{26} & C_{36} & C_{46} & C_{56} & C_{66} 
\end{bmatrix}

\begin{bmatrix}
\epsilon_{1} \\
\epsilon_{2} \\
\epsilon_{3} \\
2\epsilon_{4} \\
2\epsilon_{5} \\
2\epsilon_{6} 
\end{bmatrix}
$$

In [1]:
import numpy as np
from dataclasses import dataclass, field


@dataclass
class ElasticTensor:
    """A class that represents an elastic tensor and calculate, store and
    print various derived properties."""

    Cij: np.ndarray  # stiffness tensor in GPa
    density: float  # density in g/cm3
    Sij: np.ndarray = field(init=False)  # compliance tensor
    K_voigt: float = field(init=False)
    K_reuss: float = field(init=False)
    K_hill: float = field(init=False)
    G_voigt: float = field(init=False)
    G_reuss: float = field(init=False)
    G_hill: float = field(init=False)
    universal_anisotropy: float = field(init=False)
    isotropic_poisson_ratio: float = field(init=False)
    isotropic_avg_vp: float = field(init=False)
    isotropic_avg_vs: float = field(init=False)
    isotropic_avg_vpvs: float = field(init=False)

    def __post_init__(self):
        """_summary_
        """
        # check the symmetry of the elastic tensor
        if not np.allclose(self.Cij, self.Cij.T):
            raise Exception("the elastic tensor is not symmetric!")

        # Calculate the compliance tensor
        self.Sij = np.linalg.inv(self.Cij)

        # unpack the elastic constants to make the equations easier to read
        c11, c22, c33, c44, c55, c66 = np.diag(self.Cij)
        s11, s22, s33, s44, s55, s66 = np.diag(self.Sij)
        c12, c13, c23 = self.Cij[0, 1], self.Cij[0, 2], self.Cij[1, 2]
        s12, s13, s23 = self.Sij[0, 1], self.Sij[0, 2], self.Sij[1, 2]

        # Calculate the bulk modulus Voigt average
        self.K_voigt = 1/9 * ((c11 + c22 + c33) + 2*(c12 + c23 + c13))

        # Calculate the bulk modulus Reuss average
        self.K_reuss = 1 / ((s11 + s22 + s33) + 2*(s12 + s23 + s13))

        # Calculate bulk modulus VRH average
        self.K_hill = (self.K_voigt + self.K_reuss) / 2

        # Calculate the shear modulus Voigt average
        self.G_voigt = 1/15 * ((c11 + c22 + c33) - (c12 + c23 + c13)
                               + 3*(c44 + c55 + c66))

        # Calculate the shear modulus Reuss average
        self.G_reuss = 15 / (4*(s11 + s22 + s33) - 4*(s12 + s23 + s13)
                             + 3*(s44 + s55 + s66))

        # Calculate shear modulus VRH average
        self.G_hill = (self.G_voigt + self.G_reuss) / 2

        # Calculate the Universal elastic anisotropy
        self.universal_anisotropy = (5*(self.G_voigt / self.G_reuss)
                                     + (self.K_voigt / self.K_reuss) - 6)

        # Calculate the isotropic average Poisson ratio
        self.isotropic_poisson_ratio = ((3*self.K_hill - 2*self.G_hill)
                                        / (6*self.K_hill + 2*self.G_hill))

        # calculate the isotropic average Vp
        Vp = np.sqrt((self.K_hill + 4/3 * self.G_hill) / self.density)
        self.isotropic_avg_vp = np.around(Vp, decimals=4)

        # calculate the isotropic average Vs
        Vs = np.sqrt(self.G_hill / self.density)
        self.isotropic_avg_vs = np.around(Vs, decimals=4)

        # calculate the isotropic average Vp/Vs
        self.isotropic_avg_vpvs = np.around(Vp / Vs, decimals=4)
        

    def __repr__(self):
        return str(type(self))

    def __str__(self):
        output = ""
        output += f"Density (g/cm3): {self.density:.3f}\n"
        output += f"Stiffness Tensor (Cij):\n{self.Cij}\n"
        output += "\n"
        output += "Calculated properties:\n"
        output += f"Bulk Modulus Voigt Average (GPa): {self.K_voigt:.3f}\n"
        output += f"Bulk Modulus Reuss Average (GPa): {self.K_reuss:.3f}\n"
        output += f"Bulk Modulus VRH Average (GPa): {self.K_hill:.3f}\n"
        output += f"Shear Modulus Voigt Average (GPa): {self.G_voigt:.3f}\n"
        output += f"Shear Modulus Reuss Average (GPa): {self.G_reuss:.3f}\n"
        output += f"Shear Modulus VRH Average (GPa): {self.G_hill:.3f}\n"
        output += f"Universal Elastic Anisotropy: {self.universal_anisotropy:.3f}\n"
        output += f"Isotropic Average Poisson Ratio: {self.isotropic_poisson_ratio:.3f}\n"
        output += f"Isotropic Average Vp (km/s): {self.isotropic_avg_vp:.3f}\n"
        output += f"Isotropic Average Vs (km/s): {self.isotropic_avg_vs:.3f}\n"
        output += f"Isotropic Average Vp/Vs: {self.isotropic_avg_vpvs:.3f}\n"

        return output


In [2]:
# density (g/cm3) at 1.5 GPa from Wang et al. (2015)
rho = 2.742

# alpha quartz has six independent elastic constants: C11, C33, C12, C13, C14 and C44
C11 = 87.64
C33 = 107.20
C44 = 57.94
C12 = 6.99
C13 = 11.91
C14 = -17.19

# dependent terms
C66 = 0.5 * (C11 - C12)
C22, C55, C23, C24, C56 = C11, C44, C13, -C14, C14

# Elastic stiffness tensor (in GPa) values as a Cij matrixj
Cij_aQtz = np.array(
    [[C11, C12, C13, C14, 0.0, 0.0],
    [ C12, C22, C23, C24, 0.0, 0.0],
    [ C13, C13, C33, 0.0, 0.0, 0.0],
    [ C14, C24, 0.0, C44, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, 0.0, C55, C56],
    [ 0.0, 0.0, 0.0, 0.0, C56, C66]])

# cell parameters
cell_alpha_qtz = {
    'symmetry': '-3m',
    'unit_cell': [4.914, 4.913, 5.405],
    'angles': [90.0, 90.0, 120.0],
    'ref': 'TODO'}

In [3]:
alpha_quartz = ElasticTensor(Cij_aQtz, density=rho)

In [4]:
alpha_quartz

<class '__main__.ElasticTensor'>

In [5]:
type(alpha_quartz)

__main__.ElasticTensor

In [6]:
print(alpha_quartz)

Density (g/cm3): 2.742
Stiffness Tensor (Cij):
[[ 87.64    6.99   11.91  -17.19    0.      0.   ]
 [  6.99   87.64   11.91   17.19    0.      0.   ]
 [ 11.91   11.91  107.2     0.      0.      0.   ]
 [-17.19   17.19    0.     57.94    0.      0.   ]
 [  0.      0.      0.      0.     57.94  -17.19 ]
 [  0.      0.      0.      0.    -17.19   40.325]]

Calculated properties:
Bulk Modulus Voigt Average (GPa): 38.233
Bulk Modulus Reuss Average (GPa): 37.724
Bulk Modulus VRH Average (GPa): 37.979
Shear Modulus Voigt Average (GPa): 48.019
Shear Modulus Reuss Average (GPa): 41.821
Shear Modulus VRH Average (GPa): 44.920
Universal Elastic Anisotropy: 0.755
Isotropic Average Poisson Ratio: 0.076
Isotropic Average Vp (km/s): 5.974
Isotropic Average Vs (km/s): 4.048
Isotropic Average Vp/Vs: 1.476



In [7]:
alpha_quartz.G_voigt

48.019

$$
V_P = \sqrt{\frac{K + 4/3\mu}{\rho}}
$$

In [8]:
alpha_quartz.isotropic_avg_vp

5.9744

In [9]:
Cij_wrong = np.array(
    [[C11, C12, C13, C14, 0.0, 0.0],
    [ 6.9, C22, C23, C24, 0.0, 0.0],
    [ C13, C13, C33, 0.0, 0.0, 0.0],
    [ C14, C24, 0.0, C44, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, 0.0, C55, C56],
    [ 0.0, 0.0, 0.0, 0.0, C56, C66]])

wrong_quartz = ElasticTensor(Cij_wrong, density=rho)

Exception: the elastic tensor is not symmetric!

In [11]:
import sys
from datetime import date    
today = date.today().isoformat()

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Numpy', np.__version__)

Notebook tested in 2023-04-05 using:
Python 3.10.10 | packaged by Anaconda, Inc. | (main, Mar 21 2023, 18:39:17) [MSC v.1916 64 bit (AMD64)]
Numpy 1.23.5


Any copyright is dedicated to the Public Domain.
https://creativecommons.org/publicdomain/zero/1.0/